In [1]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import measure
from skimage.transform import resize
from keras_unet.metrics import dice_coef
from keras_unet.models import custom_unet
from keras_unet.losses import jaccard_distance
from sklearn.model_selection import train_test_split
from PIL import Image
from PIL import ImageOps
import fnmatch
import nibabel as nib
import shutil

ModuleNotFoundError: No module named 'keras_unet'

In [4]:

from numpy.lib.stride_tricks import as_strided as ast
import numpy as np
import nibabel as nib
import sys
import os 
import random
from tqdm import *
import h5py
import os.path
import cv2
#import skimage.transform as ski
import numpy as np
import sys
import nibabel as nib
from tqdm import *
import scipy
print("libraries imported")

libraries imported


In [89]:
input_folder=r"C:\Users\schmi\data\NII"
output_folder = r'C:\Users\schmi\data\output'

image_folder = 'MR'
seg_folder = 'MR_K'

#names for corresponding files
oriprefix = '_M' # MR image extension
segprefix = '_K' # Kidney segmentation extension
cystprefix = '_C' # Edge-Core segmentation extension
strremove = -len(segprefix)

In [86]:
print(input_folder+'\\'+image_folder+'\\'+filename[:strremove])

C:\Users\schmi\data\NII\MR\170795_3_93_R_M_K.n


In [79]:
def read_nifti_file(path):
    nifti_image = nib.load(path)
    nib_data = nifti_image.get_fdata()
    return nib_data

FileNotFoundError: No such file or no access: 'C:\Users\schmi\data\NII\MR\161547_0_117_R_M.nii\MR\170795_3_93_R_M_K.nii'

In [83]:
data_path = r'C:\Users\schmi\data\NII\MR'
nib.load(data_path+'\\'+filename)

FileNotFoundError: No such file or no access: 'C:\Users\schmi\data\NII\MR\170795_3_93_R_M_K.nii'

In [8]:
fnames = []
for root, dirs, files in os.walk(os.path.normpath(input_folder), topdown=True):
        for name in files:
            fnames.append(os.path.join(root, name))

        print('\nFolders have been identified\n')



Folders have been identified


Folders have been identified


Folders have been identified



In [35]:
ROI_list = []
for j in range(len(fnames)):
    ROI_name = 'M'
    filename = os.path.basename(fnames[j])
    if ROI_name in filename:
        ROI_list.append(fnames[j])

In [36]:
import shutil

In [37]:
for i in range(len(ROI_list)):
    shutil.move(ROI_list[i],seg_folder )

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\schmi\\data\\NII\\101934_0_96_L_M.nii'

In [38]:
ROI_list = []
for j in range(len(fnames)):
    ROI_name = 'M'
    filename = os.path.basename(fnames[j])
    if ROI_name in filename:
        ROI_list.append(fnames[j])

In [39]:
for i in range(len(ROI_list)):
    shutil.move(ROI_list[i],image_folder )

Error: Destination path 'C:\Users\schmi\data\NII\MR\101934_0_96_L_M.nii' already exists

In [40]:
Scan = 256
Scan2 = 256

In [113]:
input_folder = r'C:\Users\schmi\data\NII'
files__ = []
for root, dirs, files in os.walk(os.path.normpath(input_folder), topdown=True):
        for name in files:
            files__.append(os.path.join(root, name))

In [114]:
print(len(files))
print(files[3])
print(files[4])

117
101934_0_96_R_C.nii
101934_0_96_R_C.npy


FileNotFoundError: No such file or no access: 'C:\Users\schmi\data\NII\MR\170795_3_93_R_M_K.nii'

In [42]:
for filename in tqdm(files):
	count+=1
	if segprefix in filename:
		#try:
			#load MR image files
			img = nib.load(input_folder+'\\'+image_folder+'\\'+filename[:strremove])
			data = img.get_data()

			#pre-process - convert to float 32 and threshold to 99th percentile
			data=np.asarray(data).astype(np.float32)
			data = data/np.percentile(data,99) * 255
			data[data>255] = 255
			
			#load kidney seg file and pre-process
			segimg = nib.load(input_folder+'/'+seg_folder+'/'+filename)
			segdata = segimg.get_data()
			segdata = np.asarray(segdata).astype(np.float32)

			#load up cyst edge file (background==0, cysts==1, edges==2)
			cystimg = nib.load(input_folder+'/'+seg_folder+'/'+filename[:strremove]+cystprefix)
			cystdata = cystimg.get_data()
			cystdata = np.asarray(cystdata).astype(np.float32)

			aslice = np.zeros(shape=[Scan,Scan2,4],dtype='float32')

			#format all to same size
			for io in range(0,np.shape(segdata)[2]): 
				#if first slice make a zero padded version of first channel on this first slice
				if io==0:
					dataslice1 = np.zeros(shape=[Scan,Scan2],dtype='float32')
				else:
					dataslice1 = cv2.resize(data[:,:,io-1], (Scan,Scan2), interpolation=cv2.INTER_CUBIC)

				dataslice2 = cv2.resize(data[:,:,io], (Scan,Scan2), interpolation=cv2.INTER_CUBIC)

				#if last slice make a zero padded version of the last channel on this last slice
				if io==np.shape(segdata)[2]-1:
					dataslice3 = np.zeros(shape=[Scan,Scan2],dtype='float32')
				else:
					dataslice3 = cv2.resize(data[:,:,io+1], (Scan,Scan2), interpolation=cv2.INTER_CUBIC)

				segslice = cv2.resize(segdata[:,:,io], (Scan,Scan2), interpolation=cv2.INTER_NEAREST)
				cystslice = cv2.resize(cystdata[:,:,io], (Scan,Scan2), interpolation=cv2.INTER_NEAREST)

				dataslice[:,:,0] = dataslice1
				dataslice[:,:,1] = dataslice2
				dataslice[:,:,2] = dataslice3
				dataslice[:,:,3] = segslice

				#save out as numpy files
				np.save(output_folder+filename[:strremove]+'slice_'+str(io)+'.npy',dataslice)
				np.save(output_folder+filename[:strremove]+'slice_'+str(io)+'_mask.npy',cystslice)

print('done')


0it [00:00, ?it/s]

done
